In [324]:
import numpy as np 

In [346]:
def quat2rot(q):
    R = np.array([[q[0]**2 + q[1]**2 - q[2]**2 - q[3]**2, 2*(q[1] * q[2] - q[0] * q[3]),         2*(q[1] * q[3] + q[0] * q[2])],
                  [2*(q[1] * q[2] + q[0] * q[3]),         q[0]**2 - q[1]**2 + q[2]**2 - q[3]**2, 2*(q[2] * q[3] - q[0] * q[1])],
                  [2*(q[1] * q[3] - q[0] * q[2]),         2*(q[2] * q[3] + q[0] * q[1]),         q[0]**2 - q[1]**2 - q[2]**2 + q[3]**2]])
    return R
                  

def quat2zyz(q):
    r23 =  (q[2] * q[3] - q[1] * q[0])
    r13 =  (q[1] * q[3] + q[2] * q[0])
    r33 =  1 - 2*(q[1]**2 + q[2] ** 2)
    r31 =  (q[1] * q[3] - q[2] * q[0])
    r32 =  (q[2] * q[3] + q[1] * q[0])
    if r33 != 0:
        y = np.arccos(r33)
        x = np.arctan2(r32, r31)
        z = np.arctan2(r23, -r13)
        
    return np.array([x,y,z])

def quaternion_to_euler(q):
    (w, x, y, z) = (q[0], q[1], q[2], q[3])
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll = np.arctan2(t0, t1)
    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    pitch = np.arcsin(t2)
    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw = np.arctan2(t3, t4)
    return np.array([roll, pitch, yaw])

def quatmult(x,y):
    return np.array([x[0]*y[0] - x[1]*y[1] - x[2]*y[2] - x[3]*y[3],
                     x[0]*y[1] + x[1]*y[0] + x[2]*y[3] - x[3]*y[2],
                     x[0]*y[2] - x[1]*y[3] + x[2]*y[0] + x[3]*y[1],
                     x[0]*y[3] + x[1]*y[2] - x[2]*y[1] + x[3]*y[0]])

def euler_to_quaternion(r):
    (yaw, pitch, roll) = (r[0], r[1], r[2])
    qz = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
    qx = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
    qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    return np.array([qw, qx, qy, qz])

In [377]:
def r_x(x): 
    return np.array([[1,0,0], [0, np.cos(x), - np.sin(x)], [0, np. sin(x), np.cos(x)]])
def r_y(x): 
    return np.array( [[np.cos(x), 0, np.sin(x)], [0,1,0], [-np. sin(x), 0 ,  np.cos(x)]])
def r_z(x): 
    return np.array([[np.cos(x), - np.sin(x), 0], [np. sin(x), np.cos(x),0], [0,0,1]])


def xyz2quat(x,y,z):
    qx = np.array([np.cos(x/2), np.sin(x/2), 0, 0])
    qy = np.array([np.cos(y/2), 0, np.sin(y/2), 0])
    qz = np.array([np.cos(z/2), 0, 0, np.sin(z/2)])
    return quatmult(qz, quatmult(qy, qx))

def zyz2quat(x,y,z):
    qx = np.array([np.cos(x/2), 0, 0, np.sin(x/2)])
    qy = np.array([np.cos(y/2), 0, np.sin(y/2), 0])
    qz = np.array([np.cos(z/2), 0, 0, np.sin(z/2)])
    return quatmult(qx, quatmult(qy, qz))

def zyz2xyz(x,y,z):
    R = r_z(x) @ r_y(y) @ r_z(z)
    x = np.arctan(- R[1,2]/R[2,2])
    y = np.arcsin(R[0,2])
    z = np.arctan(-R[0,1] /R[0,0])
    return x,y,z

def xyz2zyz(e):
    R = r_x(e[0]) @ r_y(e[1]) @ r_z(e[2])
    if R[2,2] != 0:
        y = np.arccos(R[2,2])
        z = -np.arctan2(R[2,1], R[2,0]) +np.pi
        x = np.arctan2(R[1,2], R[0,2])
        
    return np.array([x,y,z])

In [378]:
# transforms points to quaterniions and back
x = 0.5
y = 0.3 
z = 1.3
xyz2quat(*zyz2xyz(x,y,z)) #- euler_to_quaternion(zyz2xyz(x,y,z))

array([ 0.78634338,  0.02252942,  0.14773009, -0.59944336])

In [381]:
zyz2quat(z,y,x)

1.0000000000000002

In [383]:

zyz2xyz(x,y,z)

(-0.14723056121615463, 0.2623422587933126, -1.3221341750839772)

In [349]:
#define transformation from H to R³
## works

def MRP_H2R(q):
    return 4 * q[1:]/(1+q[0])

def MRP_R2H(e):
    return 1/(16 + np.linalg.norm(e)**2) * np.array([16 - np.linalg.norm(e)**2, *8*e])

In [358]:
# complex conjugated 
def quat_cc(q):
    d = np.copy(q)
    d[1:] *= -1
    return d

In [359]:
a = euler_to_quaternion(zyz2xyz(x,y,z))

In [363]:
quatmult(quat_cc(a), a)

array([ 1.00000000e+00,  0.00000000e+00, -3.46944695e-18,  0.00000000e+00])

In [361]:
quat_cc(a)

array([ 0.78634338, -0.02252942, -0.14773009,  0.59944336])

In [364]:
MRP_H2R(np.array([1,0,0,0]))

array([0., 0., 0.])

In [365]:
import sympy as s
def r_x(x): 
    return np.array([[1,0,0], [0, s.cos(x), - s.sin(x)], [0, s.sin(x), s.cos(x)]])
def r_y(x): 
    return np.array( [[s.cos(x), 0, s.sin(x)], [0,1,0], [-s.sin(x), 0 ,  s.cos(x)]])
def r_z(x): 
    return np.array([[s.cos(x), - s.sin(x), 0], [s.sin(x), s.cos(x),0], [0,0,1]])



In [367]:
x = s.Symbol('zyz[0]')
y = s.Symbol('zyz[1]')
z = s.Symbol('zyz[2]')
r_z(x) @ r_y(y) @ r_z(z)

array([[-sin(zyz[0])*sin(zyz[2]) + cos(zyz[0])*cos(zyz[1])*cos(zyz[2]),
        -sin(zyz[0])*cos(zyz[2]) - sin(zyz[2])*cos(zyz[0])*cos(zyz[1]),
        sin(zyz[1])*cos(zyz[0])],
       [sin(zyz[0])*cos(zyz[1])*cos(zyz[2]) + sin(zyz[2])*cos(zyz[0]),
        -sin(zyz[0])*sin(zyz[2])*cos(zyz[1]) + cos(zyz[0])*cos(zyz[2]),
        sin(zyz[0])*sin(zyz[1])],
       [-sin(zyz[1])*cos(zyz[2]), sin(zyz[1])*sin(zyz[2]), cos(zyz[1])]],
      dtype=object)